In [1]:
from transformers import TFAutoModel
bert = TFAutoModel.from_pretrained('bert-base-cased')
ddir = './data'
mdir = './models'

2023-03-20 04:30:38.352570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 04:30:38.992634: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-20 04:30:38.992689: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-20 04:30:38.992695: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [3]:
import tensorflow as tf

In [4]:
input_ids = tf.keras.layers.Input(shape = (512,), name = 'input_ids', dtype = 'int32')
mask = tf.keras.layers.Input(shape = (512,), name = 'attention_mask', dtype = 'int32')

In [5]:
embeddings = bert.bert(input_ids, attention_mask = mask)[1]

In [6]:
x = tf.keras.layers.Dense(1024, activation = 'relu') (embeddings)
y = tf.keras.layers.Dense(5, activation = 'softmax', name = 'outputs') (x)

In [7]:
model = tf.keras.Model(inputs = [input_ids, mask], outputs = y)

In [8]:
model.layers[2].trainable = False

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5, weight_decay = 1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer = optimizer, loss = loss, metrics = [acc])

In [11]:
element_spec = ({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
                 'attention_mask': tf.TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
                tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [12]:
train_ds =tf.data.Dataset.load(f'{ddir}train', element_spec = element_spec)
val_ds =tf.data.Dataset.load(f'{ddir}val', element_spec = element_spec)
train_ds.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [13]:
tf.test.is_built_with_cuda()

True

In [14]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf. __version__) 
tf.config.experimental.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2447407623973246559
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5237112832
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16100187760674063740
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]
2.11.0


2023-03-20 04:31:18.238818: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 04:31:18.239040: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 04:31:18.239168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 04:31:18.239328: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-20 04:31:18.239453: I tensorflow/compiler/xla/stream_executo

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 3
)

Epoch 1/3


2023-03-20 04:31:28.550067: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f0ca000b640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-20 04:31:28.550089: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1060, Compute Capability 6.1
2023-03-20 04:31:28.554971: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-20 04:31:28.622935: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-20 04:31:28.659157: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8778/8778 [==============================] - 9869s 1s/step - loss: 1.1131 - accuracy: 0.5530 - val_loss: 1.0794 - val_accuracy: 0.5639
Epoch 2/3
8778/8778 [==============================] - 9611s 1s/step - loss: 1.0492 - accuracy: 0.5734 - val_loss: 1.0556 - val_accuracy: 0.5719
Epoch 3/3
8778/8778 [==============================] - 10384s 1s/step - loss: 1.0319 - accuracy: 0.5793 - val_loss: 1.0424 - val_accuracy: 0.5772


In [16]:
model.save(f'{mdir}sentiment_model_local')

INFO:tensorflow:Assets written to: sentiment_model_local/assets


INFO:tensorflow:Assets written to: sentiment_model_local/assets
